In [1]:
DATA_PATH = "Data/"

# Packages

In [2]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from tqdm import tqdm
from math import ceil

In [3]:
import tensorflow as tf

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Reshape
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import load_img

2021-08-31 13:49:29.025437: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
# Size and Boarder of the images
n_classes = 4
im_width = 512
im_height = 512
border = 0

# Define Model

In [5]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

In [6]:
def get_unet(input_img, n_classes = 1,n_filters=16, dropout=0.1, batchnorm=True):
    """Function to define the UNET Model"""
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters * 2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters * 4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters * 8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)

    c5 = conv2d_block(p4, n_filters=n_filters * 16, kernel_size=3, batchnorm=batchnorm)

    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size=3, batchnorm=batchnorm)

    #c10 = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
    c10 = Conv2D(n_classes, (1, 1), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c9)
    c10 = Reshape((im_height * im_width, n_classes))(c10)

    outputs = Activation('softmax')(c10)
    
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [7]:
#with strategy.scope():
input_img = Input((im_height, im_width, 3), name='img')
model = get_unet(input_img, n_classes = n_classes,n_filters=16, dropout=0.05, batchnorm=True)
model.compile(optimizer="adam", loss= "categorical_crossentropy", metrics=["accuracy"], sample_weight_mode = 'temporal')
callbacks = [
    EarlyStopping(patience=100, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=30, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-fog_pato.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

2021-08-31 13:49:29.997612: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-31 13:49:29.999064: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-31 13:49:30.465357: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 13:49:30.466142: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-08-31 13:49:30.466585: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node ze

# Collect Images

In [8]:
ids = []
l = os.listdir(DATA_PATH+"image_tiles/")
l.sort()
for filename in l:
    if filename.endswith(".png"):
        ids.append(filename)
print("No. of images = ", len(ids))

No. of images =  6251


In [11]:
model.load_weights('model-fog_pato.h5') 
n_classes = 3

b_size = 100
n = 0
while n < ceil(len(ids)/b_size):
    print(n)
    X = np.zeros((b_size, im_height, im_width, 3), dtype=np.float32)
    
    lower_bound = n*b_size
    if (n+1)*b_size > len(ids):
        upper_bound = len(ids)
    else:
        upper_bound = (n+1)*b_size
        
    nn = 0
    for i in ids[lower_bound:upper_bound]:
        img = load_img(f"{DATA_PATH}image_tiles/{i}")
        x_img = np.array(img)
        X[nn] = x_img/255.0
        nn += 1
    preds = model.predict(X, verbose=1)
    h = im_width
    w = im_height
    preds = preds.reshape((b_size,h,w,n_classes+1))
    preds_con = np.zeros((b_size,h,w,n_classes))
    preds_con[:,:,:,0] = preds[:,:,:,0]
    preds_con[:,:,:,1] = preds[:,:,:,1] + preds[:,:,:,2]
    preds_con[:,:,:,2] = preds[:,:,:,3]
    del preds

    nn = 0
    for i in ids[lower_bound:upper_bound]:
        cv.imwrite(f"{DATA_PATH}predicted_tiles/{i}",cv.cvtColor((preds_con[nn]*255).astype(np.uint8), cv.COLOR_RGB2BGR))
        nn += 1
    n += 1

0
4/4 [==============================] - 1s 390ms/step
1
4/4 [==============================] - 1s 368ms/step
2
4/4 [==============================] - 1s 367ms/step
3
4/4 [==============================] - 1s 361ms/step
4
4/4 [==============================] - 1s 357ms/step
5
4/4 [==============================] - 1s 358ms/step
6
4/4 [==============================] - 1s 359ms/step
7
4/4 [==============================] - 1s 371ms/step
8
4/4 [==============================] - 1s 370ms/step
9
4/4 [==============================] - 1s 362ms/step
10
4/4 [==============================] - 1s 358ms/step
11
4/4 [==============================] - 1s 358ms/step
12
4/4 [==============================] - 1s 362ms/step
13
4/4 [==============================] - 1s 373ms/step
14
4/4 [==============================] - 1s 371ms/step
15
4/4 [==============================] - 1s 362ms/step
16
4/4 [==============================] - 1s 361ms/step
17
4/4 [==============================] - 1s 356ms/step
18